In [ ]:
import os
import json
import traceback
import pandas as pd

In [35]:
import langchain

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [37]:
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
KEY=os.getenv("GOOGLE_API_KEY")


In [39]:
llm=ChatGoogleGenerativeAI(google_api_key=KEY,model="gemini-2.5-flash",temperature=0.5, max_output_tokens=1500)

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [41]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here", 
        },
        "correct": "correct answer",
    },
}

In [42]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [43]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [44]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

In [45]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
and if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits
the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [46]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [47]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [48]:
generate_evaluate_chains=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"],
                                         output_variables=["quiz","review"], verbose=True,)


In [49]:
file_path=r"E:\AI\data.txt"

In [50]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [51]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contribu

In [52]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [53]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [54]:

response=generate_evaluate_chains(
        {
            "text":TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

In [67]:
quiz=response.get("quiz")

In [71]:
import ast

# remove code fences
import re
cleaned = re.sub(r"^```json\n", "", quiz)
cleaned = re.sub(r"\n```$", "", cleaned)

data = ast.literal_eval(cleaned)   # handles single quotes, dict-style
print(data.keys())


dict_keys(['1', '2', '3', '4', '5'])


In [73]:
quiz=data

In [74]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq= value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq,"Choices": options, "Correct": correct})

In [75]:
quiz_table_data

[{'MCQ': "Who is credited with coining the term 'machine learning' and in what year?",
  'Choices': 'a: Donald Hebb in 1949 | b: Arthur Samuel in 1959 | c: Tom M. Mitchell in 1981 | d: Walter Pitts in the 1950s',
  'Correct': 'b'},
 {'MCQ': "Which experimental 'learning machine' was developed by Raytheon Company in the early 1960s to analyze signals using rudimentary reinforcement learning?",
  'Choices': "a: The Hebbian Learner | b: The Samuel's Checker | c: Cybertron | d: The Turing Machine",
  'Correct': 'c'},
 {'MCQ': 'According to the text, what are the two main objectives of modern-day machine learning?',
  'Choices': 'a: To play computer games and write books | b: To study human cognitive processes and invent new algorithms | c: To classify data based on models and make predictions for future outcomes | d: To analyze sonar signals and electrocardiograms',
  'Correct': 'c'},
 {'MCQ': "Which Canadian psychologist published 'The Organization of Behavior' in 1949, introducing a theo

In [77]:
quiz=pd.DataFrame(quiz_table_data)

In [78]:
quiz.to_csv("machinelearning.csv",index=False)